In [28]:
import getpass
import pandas as pd
import requests
import pandas as pd
import googlemaps
from itertools import tee

## Getting distance and time from Google API 

Hypothesis: the travel time from a house to the city center is an important Factor that determines the price of a house

Therefor the traveltime from the house to the city center of Seatle is needed. This can be accessed through Google Distance Matrix API

## 1) Save API Access Token

In [1]:

YOUR_API_KEY = getpass.getpass(prompt='YOUR_API_KEY:'"")

YOUR_API_KEY:········


## 2) Example request to API and respons

In [2]:
#url = "https://maps.googleapis.com/maps/api/distancematrix/json?origins=Washington%2C%20DC&destinations=New%20York%20City%2C%20NY&units=imperial&key="+YOUR_API_KEY

#payload={}
#headers = {}

#response = requests.request("GET", url, headers=headers, data=payload)

#print(response.text)

{
   "destination_addresses" : [ "New York, NY, USA" ],
   "origin_addresses" : [ "Washington, DC, USA" ],
   "rows" : [
      {
         "elements" : [
            {
               "distance" : {
                  "text" : "225 mi",
                  "value" : 361947
               },
               "duration" : {
                  "text" : "3 hours 56 mins",
                  "value" : 14145
               },
               "status" : "OK"
            }
         ]
      }
   ],
   "status" : "OK"
}



## 3) Import Dataframe

In [9]:
df = pd.read_csv('C:/Users/Lenovo/Documents/GitHub/Houseprices/Data/house_price_df.csv')
df_latlong = df[['id', 'lat', 'long']]
df_latlong

,id,lat,long
0,7129300520,47.5112,-122.257
1,6414100192,47.7210,-122.319
2,5631500400,47.7379,-122.233
3,2487200875,47.5208,-122.393
4,1954400510,47.6168,-122.045
...,...,...,...
21592,263000018,47.6993,-122.346
21593,6600060120,47.5107,-122.362
21594,1523300141,47.5944,-122.299
21595,291310100,47.5345,-122.069


In [49]:
df_latlong.shape

(21597, 5)

In [50]:
df.shape

(21597, 22)

## 4) Getting Coordinates of Seattle City Center (Space Needle)

In [10]:
Seattle_Center = [47.62023993864965, -122.34923129807153]

## 5) Adding Seatle_Center to df_latlong

In [12]:
df_latlong[['Center_lat', 'Center_long']] = Seattle_Center

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_904\1671102843.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_latlong[['Center_lat', 'Center_long']] = Seattle_Center
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_904\1671102843.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_latlong[['Center_lat', 'Center_long']] = Seattle_Center


In [13]:
df_latlong

,id,lat,long,Center_lat,Center_long
0,7129300520,47.5112,-122.257,47.62024,-122.349231
1,6414100192,47.7210,-122.319,47.62024,-122.349231
2,5631500400,47.7379,-122.233,47.62024,-122.349231
3,2487200875,47.5208,-122.393,47.62024,-122.349231
4,1954400510,47.6168,-122.045,47.62024,-122.349231
...,...,...,...,...,...
21592,263000018,47.6993,-122.346,47.62024,-122.349231
21593,6600060120,47.5107,-122.362,47.62024,-122.349231
21594,1523300141,47.5944,-122.299,47.62024,-122.349231
21595,291310100,47.5345,-122.069,47.62024,-122.349231


## 6) Create loop

In [32]:
#LatOrigin = row1['Latitude'] 
#LongOrigin = row1['Longitude']
#origins = (LatOrigin,LongOrigin)

origins = tuple(df_latlong[['lat', 'long']].iloc[0,:])
origins

(47.5112, -122.257)

In [33]:
destination = tuple(df_latlong[['Center_lat','Center_long']].iloc[0,:])
destination

(47.62023993864965, -122.34923129807153)

In [29]:
gmaps = googlemaps.Client(key=YOUR_API_KEY)

In [38]:
list = [0]

In [44]:
for i in range(0,len(df)):
    origins = tuple(df_latlong[['lat', 'long']].iloc[i,:])
    destination = tuple(df_latlong[['Center_lat','Center_long']].iloc[i,:])
    result = gmaps.distance_matrix(origins, destination, mode='driving')["rows"][0]["elements"][0]["distance"]["value"]
    list.append(result)

In [47]:
len(list)

21698

In [54]:
len(list[101:])

21597

In [55]:
df_latlong['DistanceM'] = list[101:]

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_904\1249308983.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_latlong['DistanceM'] = list[101:]


In [56]:
df_latlong

,id,lat,long,Center_lat,Center_long,DistanceM
0,7129300520,47.5112,-122.257,47.62024,-122.349231,18112
1,6414100192,47.7210,-122.319,47.62024,-122.349231,14155
2,5631500400,47.7379,-122.233,47.62024,-122.349231,25519
3,2487200875,47.5208,-122.393,47.62024,-122.349231,17950
4,1954400510,47.6168,-122.045,47.62024,-122.349231,33624
...,...,...,...,...,...,...
21592,263000018,47.6993,-122.346,47.62024,-122.349231,9458
21593,6600060120,47.5107,-122.362,47.62024,-122.349231,15749
21594,1523300141,47.5944,-122.299,47.62024,-122.349231,8836
21595,291310100,47.5345,-122.069,47.62024,-122.349231,30793


In [ ]:
"""def pairwise(iterable):
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

#empty list - will be used to store calculated distances
list = [0]

# Loop through each row in the data frame using pairwise
for (i1, row1), (i2, row2) in pairwise(df.iterrows()):
      #Assign latitude and longitude as origin/departure points
      LatOrigin = row1['Latitude'] 
      LongOrigin = row1['Longitude']
      origins = (LatOrigin,LongOrigin)

      #Assign latitude and longitude from the next row as the destination point
      LatDest = row2['Latitude']   # Save value as lat
      LongDest = row2['Longitude'] # Save value as lat
      destination = (LatDest,LongDest)

      #pass origin and destination variables to distance_matrix function# output in meters
     
      #append result to list
      list.append(result)"""

In [59]:
df_latlong.to_csv('C:/Users/Lenovo/Documents/GitHub/Houseprices/Data/distance.csv')

In [61]:
pd.DataFrame(list).to_csv('C:/Users/Lenovo/Documents/GitHub/Houseprices/Data/listorg.csv')